<a href="https://colab.research.google.com/github/Diksha-Arsule/Wad/blob/main/prac4ecg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# Step a) Import required libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers, optimizers

In [23]:
from google.colab import files
uploaded = files.upload()

Saving ecg.csv to ecg (1).csv


In [25]:
df = df.apply(pd.to_numeric, errors='coerce')
X = df.values

In [26]:
# Step d) Check shape of dataset
print("Shape of ECG data:", X.shape)

Shape of ECG data: (4997, 141)


In [27]:
# Step g) Scale the data using MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [28]:
# Step e) Split into train and test
X_train, X_test = train_test_split(X_scaled, test_size=0.2, shuffle=True)

In [29]:
# Step h & i) Create Autoencoder Model (Encoder + Decoder)
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Encoder Layer 1
    layers.Dense(32, activation='relu'),                                   # Encoder Layer 2 (latent space)
    layers.Dense(64, activation='relu'),                                   # Decoder Layer 1
    layers.Dense(X_train.shape[1], activation='sigmoid')                   # Decoder Layer 2 (output layer)
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
# Step j) Compile model
model.compile(
    optimizer=optimizers.Adam(),
    loss='mean_squared_logarithmic_error'
)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         9,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 141)            │         9,165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,445 (87.68 KB)

 Trainable params: 22,445 (87.68 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
# Train model
history = model.fit(
    X_train, X_train,          # Autoencoder tries to reproduce its own input
    epochs=20,
    batch_size=512,
    validation_split=0.2
)

Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 112ms/step - loss: 0.0129 - val_loss: 0.0113
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0110 - val_loss: 0.0091
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0088 - val_loss: 0.0071
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0069 - val_loss: 0.0057
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0057 - val_loss: 0.0050
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0051 - val_loss: 0.0047
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0048 - val_loss: 0.0044
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0045 - val_loss: 0.0042
Epoch 9/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0043 - val_loss: 0.0039
Epoch 10/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0040 - val_loss: 0.0035
Epoch 11/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0035 - val_loss: 0.0031
Epoch 12/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0030 - val_loss: 0.0026


In [32]:
# Step: Compute Reconstruction Errors
reconstructed = model.predict(X_test)
errors = np.mean(np.abs(reconstructed - X_test), axis=1)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [33]:
# Print sample reconstruction errors
print("\nSample Reconstruction Errors:")
print(errors[:10])


Sample Reconstruction Errors:
[0.05634648 0.02977737 0.03846833 0.02546686 0.02056771 0.02358827
 0.04067421 0.02409311 0.05731704 0.01970406]


In [34]:
# Step: Determine anomaly threshold (Mean + Standard Deviation)
threshold = errors.mean() + errors.std()
print("\nAnomaly Detection Threshold:", threshold)


Anomaly Detection Threshold: 0.06945445153116021


In [35]:
# Step: Detect anomalies (True = Anomaly, False = Normal)
anomaly_flags = errors > threshold
print("\nAnomaly Flags for first 20 test samples (True means anomaly):")
print(anomaly_flags[:20])


Anomaly Flags for first 20 test samples (True means anomaly):
[False False False False False False False False False False False False
 False  True False False False False False False]
